# Clustering Methods

## Introduction

The goal of this notebook is to provide an overview of two traditional clustering algorithms available in R, namely K-Means and Hierarchical clustering. In order to showcase the different methods, we are going to apply them to a [Mice Protein Expression Data Set](https://archive.ics.uci.edu/ml/datasets/Mice+Protein+Expression#) available from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.html).

The actual dataset contains the expression levels of 77 proteins measured in the cerebral cortex of 8 classes of control and Down syndrome mice exposed to context fear conditioning.

## Dataset information
*(information copied from the Dataset url)*   
The data set consists of the expression levels of 77 proteins/protein modifications that produced detectable signals in the nuclear fraction of cortex. There are 38 control mice and 34 trisomic mice (Down syndrome), for a total of 72 mice. In the experiments, 15 measurements were registered of each protein per sample/mouse. Therefore, for control mice, there are 38x15, or 570 measurements, and for trisomic mice, there are 34x15, or 510 measurements. The dataset contains a total of 1080 measurements per protein. Each measurement can be considered as an independent sample/mouse. 

The eight classes of mice are described based on features such as genotype, behavior and treatment. According to genotype, mice can be control or trisomic. According to behavior, some mice have been stimulated to learn (context-shock) and others have not (shock-context) and in order to assess the effect of the drug memantine in recovering the ability to learn in trisomic mice, some mice have been injected with the drug and others have not. 

**Classes**
- c-CS-s: control mice, stimulated to learn, injected with saline (9 mice) 
- c-CS-m: control mice, stimulated to learn, injected with memantine (10 mice) 
- c-SC-s: control mice, not stimulated to learn, injected with saline (9 mice) 
- c-SC-m: control mice, not stimulated to learn, injected with memantine (10 mice) 
- t-CS-s: trisomy mice, stimulated to learn, injected with saline (7 mice) 
- t-CS-m: trisomy mice, stimulated to learn, injected with memantine (9 mice) 
- t-SC-s: trisomy mice, not stimulated to learn, injected with saline (9 mice) 
- t-SC-m: trisomy mice, not stimulated to learn, injected with memantine (9 mice) 

The aim is to identify subsets of proteins that are discriminant between the classes. 

### Attribute information

**Column 1**: _Mouse ID_  
**Columns 2..78**: _Values of expression levels of 77 proteins_; the names of proteins are followed by \_ indicating that they were measured in the nuclear fraction. For example: DYRK1A_n  
**Column 79**: _Genotype_: control (c) or trisomy (t)  
**Column 80**: _Treatment type_: memantine (m) or saline (s)   
**Column 81**: _Behavior_: context-shock (CS) or shock-context (SC)   
**Column 82**: _Class_: c-CS-s, c-CS-m, c-SC-s, c-SC-m, t-CS-s, t-CS-m, t-SC-s, t-SC-m   



## Loading the data
For your convenience, and in order to avoid any potential formatting issues due to the xls format available, we provide you with the entire dataset in csv format. We assume that you have downloaded the file and it now exists within the folder `data`. Moreover, we add the sep option because the default separator is the empty string. Finally, we are also going to import two libraries, `dplyr` and `gplots` which will help us clean-up the data and plot any figures respectively.


In [1]:
library(dplyr);
library(gplots);
MouseDataRaw <- read.csv(file="data/Data_Cortex_Nuclear.csv",head=TRUE,sep=";");


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
: package 'gplots' was built under R version 3.2.4
Attaching package: 'gplots'

The following object is masked from 'package:stats':

    lowess



We can use the `names()` command, in order to verify that the data (and the attributes) have been loaded correctly.

In [2]:
names(MouseDataRaw);

[1] "MouseID"         "DYRK1A_N"        "ITSN1_N"         "BDNF_N"         
 [5] "NR1_N"           "NR2A_N"          "pAKT_N"          "pBRAF_N"        
 [9] "pCAMKII_N"       "pCREB_N"         "pELK_N"          "pERK_N"         
[13] "pJNK_N"          "PKCA_N"          "pMEK_N"          "pNR1_N"         
[17] "pNR2A_N"         "pNR2B_N"         "pPKCAB_N"        "pRSK_N"         
[21] "AKT_N"           "BRAF_N"          "CAMKII_N"        "CREB_N"         
[25] "ELK_N"           "ERK_N"           "GSK3B_N"         "JNK_N"          
[29] "MEK_N"           "TRKA_N"          "RSK_N"           "APP_N"          
[33] "Bcatenin_N"      "SOD1_N"          "MTOR_N"          "P38_N"          
[37] "pMTOR_N"         "DSCR1_N"         "AMPKA_N"         "NR2B_N"         
[41] "pNUMB_N"         "RAPTOR_N"        "TIAM1_N"         "pP70S6_N"       
[45] "NUMB_N"          "P70S6_N"         "pGSK3B_N"        "pPKCG_N"        
[49] "CDK5_N"          "S6_N"            "ADARB1_N"        "AcetylH3K9_N"   
[53] "RRP1_N"          "BAX_N"           "ARC_N"           "ERBB4_N"        
[57] "nNOS_N"          "Tau_N"           "GFAP_N"          "GluR3_N"        
[61] "GluR4_N"         "IL1B_N"          "P3525_N"         "pCASP9_N"       
[65] "PSD95_N"         "SNCA_N"          "Ubiquitin_N"     "pGSK3B_Tyr216_N"
[69] "SHH_N"           "BAD_N"           "BCL2_N"          "pS6_N"          
[73] "pCFOS_N"         "SYP_N"           "H3AcK18_N"       "EGR1_N"         
[77] "H3MeK4_N"        "CaNA_N"          "Genotype"        "Treatment"      
[81] "Behavior"        "class"

Also, we can use the `summary()` command to get a quick overview of the data content.

In [3]:
summary(MouseDataRaw);

     MouseID        DYRK1A_N         ITSN1_N           BDNF_N      
 18899_1 :   1   Min.   :0.1453   Min.   :0.2454   Min.   :0.1152  
 18899_10:   1   1st Qu.:0.2881   1st Qu.:0.4734   1st Qu.:0.2874  
 18899_11:   1   Median :0.3664   Median :0.5658   Median :0.3166  
 18899_12:   1   Mean   :0.4258   Mean   :0.6171   Mean   :0.3191  
 18899_13:   1   3rd Qu.:0.4877   3rd Qu.:0.6980   3rd Qu.:0.3482  
 18899_14:   1   Max.   :2.5164   Max.   :2.6027   Max.   :0.4972  
 (Other) :1074   NA's   :3        NA's   :3        NA's   :3       
     NR1_N           NR2A_N          pAKT_N           pBRAF_N       
 Min.   :1.331   Min.   :1.738   Min.   :0.06324   Min.   :0.06404  
 1st Qu.:2.057   1st Qu.:3.156   1st Qu.:0.20575   1st Qu.:0.16459  
 Median :2.297   Median :3.761   Median :0.23118   Median :0.18230  
 Mean   :2.297   Mean   :3.844   Mean   :0.23317   Mean   :0.18185  
 3rd Qu.:2.528   3rd Qu.:4.440   3rd Qu.:0.25726   3rd Qu.:0.19742  
 Max.   :3.758   Max.   :8.483   Max.   :0

## Construct clean dataset

The next step, before moving on to the clustering process, it would be best if we "clean-up" the data. Therefore, we are going to remove the columns that would not be necessary for the clustering. In order to identify these columns, we need to decide first on what would be the level of differentiation; `Genotype`, `Treatment`, `Behavior` or `Class`. In this notebook, we are going to use the `Class` as the level, and therefore remove attributes `MouseID`, `Genotype`, `Treatment` and `Behavior`. Moreover, in order to produce a well-formatted dataset, we are going to rename each row with a new identifier that is constructed by the `MouseID` and the `Class` information.

First, include an extra column that will correspond to the label of each instance. The label is constructed through concantenation of two existing fields: MouseID and class  (e.g.: `309_1 c-CS-m` )

In [4]:
MouseDataRaw$rowNamesInfo <- paste(MouseDataRaw$MouseID, MouseDataRaw$class, sep="   ");

Secondly, we will temporarily split the dataset at the different levels (i.e. 8 subsets if we use the `Class` attribute). Each subset will contain 78 attributes: the 77 protein expression levels, and the label we just created. Finally, any missing values will also be omitted. It is important to note that this step can be modified to different levels, as well as any application of normalization (if required).

In [5]:
MouseData_cCSs <- MouseDataRaw %>%
  na.omit() %>%
  filter(class == "c-CS-s") %>%
  select(-MouseID, -Genotype, -Treatment, -Behavior);


MouseData_cCSm <- MouseDataRaw %>%
  na.omit() %>%
  filter(class == "c-CS-m") %>%
  select(-MouseID, -Genotype, -Treatment, -Behavior);

MouseData_cSCs <- MouseDataRaw %>%
  na.omit() %>%
  filter(class == "c-SC-s") %>%
  select(-MouseID, -Genotype, -Treatment, -Behavior);

MouseData_cSCm <- MouseDataRaw %>%
  na.omit() %>%
  filter(class == "c-SC-m") %>%
  select(-MouseID, -Genotype, -Treatment, -Behavior);

MouseData_tCSs <- MouseDataRaw %>%
  na.omit() %>%
  filter(class == "t-CS-s") %>%
  select(-MouseID, -Genotype, -Treatment, -Behavior);

MouseData_tCSm <- MouseDataRaw %>%
  na.omit() %>%
  filter(class == "t-CS-m") %>%
  select(-MouseID, -Genotype, -Treatment, -Behavior);

MouseData_tSCs <- MouseDataRaw %>%
  na.omit() %>%
  filter(class == "t-SC-s") %>%
  select(-MouseID, -Genotype, -Treatment, -Behavior);

MouseData_tSCm <- MouseDataRaw %>%
  na.omit() %>%
  filter(class == "t-SC-m") %>%
  select(-MouseID, -Genotype, -Treatment, -Behavior);

The last step in this process will be to join all subsets for the final, clean dataset. The label attribute is assigned as the row name, and dropped as an independent attribute. Ultimately, each row of the final dataset will have exactly 77 attributes (one for each of the 77 proteins).

In [6]:
MouseDataClean <- bind_rows(MouseData_cCSs, MouseData_cCSm, MouseData_cSCs, MouseData_cSCm, MouseData_tCSs, MouseData_tCSm, MouseData_tSCs, MouseData_tSCm);
rownames(MouseDataClean) <- MouseDataClean$rowNamesInfo; 
MouseData <- select(MouseDataClean, -rowNamesInfo, -class);

Finally, you can use `head()` and `summary()` once again, to get an overview of the dataset in its current form.

In [7]:
head(MouseData);
summary(MouseData);

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,ellip.h,SHH_N,BAD_N,BCL2_N,pS6_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N
1,0.4458384,0.719069,0.4190171,2.859232,5.321076,0.229538,0.1712234,3.518429,0.2241737,1.502336,<8b>,0.2052275,0.1366628,0.1069778,0.1252275,0.1149125,0.5784831,0.195846,0.1494049,0.1824971,1.736803
2,0.4274166,0.7232267,0.4328929,2.939673,5.38491,0.2344402,0.169854,3.541551,0.2389604,1.575278,<8b>,0.2247601,0.135684,0.1210247,0.132908,0.1245799,0.5950421,0.2085423,0.1612039,0.1939317,1.84391
3,0.4567857,0.7507313,0.4632568,3.090683,5.576101,0.2443046,0.1782643,3.654995,0.235972,1.722099,<8b>,0.2295513,0.1298929,0.1298468,0.1287389,0.1326625,0.5667467,0.2008863,0.1651588,0.1794221,1.77031
4,0.3662387,0.5892331,0.3644717,2.534339,4.605254,0.2343032,0.1873012,3.230416,0.194487,1.342561,<8b>,0.2057434,0.1421907,0.1097177,0.1277717,0.1154732,0.556222,0.2028353,0.1605477,0.1905974,1.763359
5,0.3851905,0.6067402,0.3761718,2.584431,4.786994,0.2408924,0.1698113,3.230806,0.1905779,1.457933,<8b>,0.22147,0.1412626,0.1201586,0.1329674,0.1230253,0.5512046,0.2114059,0.1664532,0.1915828,1.807502
6,0.3773878,0.6306113,0.3909981,2.666428,5.101839,0.2443887,0.1766953,3.27149,0.1948424,1.531519,<8b>,0.2264863,0.1370762,0.1228518,0.1285416,0.1240711,0.525836,0.2072689,0.167615,0.1916512,1.707095


    DYRK1A_N         ITSN1_N           BDNF_N           NR1_N      
 Min.   :0.1453   Min.   :0.2454   Min.   :0.1152   Min.   :1.331  
 1st Qu.:0.2908   1st Qu.:0.4805   1st Qu.:0.2790   1st Qu.:2.044  
 Median :0.3721   Median :0.5903   Median :0.3085   Median :2.285  
 Mean   :0.4152   Mean   :0.6231   Mean   :0.3150   Mean   :2.295  
 3rd Qu.:0.4957   3rd Qu.:0.7306   3rd Qu.:0.3470   3rd Qu.:2.544  
 Max.   :0.9922   Max.   :1.3364   Max.   :0.4972   Max.   :3.758  
     NR2A_N          pAKT_N          pBRAF_N         pCAMKII_N    
 Min.   :1.738   Min.   :0.1210   Min.   :0.1076   Min.   :1.344  
 1st Qu.:3.118   1st Qu.:0.1974   1st Qu.:0.1567   1st Qu.:2.484  
 Median :3.708   Median :0.2206   Median :0.1761   Median :3.370  
 Mean   :3.784   Mean   :0.2245   Mean   :0.1754   Mean   :3.621  
 3rd Qu.:4.343   3rd Qu.:0.2478   3rd Qu.:0.1925   3rd Qu.:4.596  
 Max.   :8.483   Max.   :0.3538   Max.   :0.3171   Max.   :7.105  
    pCREB_N           pELK_N          pERK_N           

## K-Means clustering

In order to perform k-means clustering, we need to select:
1. the number of centers `k`,  
2. the number of random starting centers `nstart`,  
3. the specific algorithm to be used as the underlying method (choices are `"Hartigan-Wong"` (default), `"Lloyd"` and `"MacQueen"`,  and 
4. the specific attributes (for visualizion purposes)    
that will better facilitate clustering of instances. You should investigate the best possible choice for these two options. In our case, we will set `k=8`.

To check the documentation of the `kmeans` function use the command `?kmeans`.

In [8]:
kmeansK = 8;
clusters <- kmeans(MouseData, centers = kmeansK, nstart = 15, algorithm = "Hartigan-Wong");

In order to visualize the results of the clustering, we will create a PNG file, and select two attributes for the 2D plot axes; `DYRK1A_N` and `BDNF_N` protein expressions (i.e. attributes \#1 and \#3). Moreover, we will plot a cross on the cluster centers (as projected on the particular 2D plot), that identifies the centers for the selected columns (attributes).

In [9]:
png("KmeansMouseProteinExpression.png", # create PNG for the plot        
    width     = 8000,                   # set the width of the image in pixels
    height    = 6000,                   # set the height of the image in pixels
    res       = 300,                    # set the resolutions to 300 pixels per inch
    pointsize = 18);                     # set the size of any letters/text

# Plots the attributes DYRK1A_N and BDNF_N (columns 1 and 3 respectively) using the cluster id as different colors
plot(MouseData$DYRK1A_N, MouseData$BDNF_N, col = clusters$cluster);

# Identifies the centers for the selected columns (attributes), i.e. 1 and 3
centers <-subset(clusters$centers, select = c(1,3));

# Prints out the centroids as crosses on the figure
points(centers, col = 1:kmeansK, pch= 3, cex= 3, lwd= 3);

legend("topright",            # location of the legend on the heatmap plot
       legend = 1:kmeansK,    # category labels
       col = 1:kmeansK,       # 
       lty= 1,                # line style
       lwd = 10               # line width
);

# Closes the devices (i.e. the "print to PNG" process)
dev.off()

pdf 
  2

The output of the previous commands is the figure shown below:

![image](KmeansMouseProteinExpression.png)

It is obvious that the figure by itself is not informative (although a different selection of attributes may provide better insights into the issue). However, we can use the Sum of Squared Errors (SSE) in order to have a quantitative measure of the clustering. We need the SSE between clusters to be high, whereas the SSE witin clusters to be low.

In [10]:
cat("\nSEE Results:\n===\n\n")
cat(paste("  SSE Between Clusters  : ", clusters$betweenss, "\n",sep=" "));
cat(paste("SSE Total within-cluster: ", clusters$tot.withinss, "\n",sep=" "));
cat(paste("      SSE Total         : ", clusters$totss, "\n",sep=" "));


SEE Results:
===

  SSE Between Clusters  :  2017.59368441346 
SSE Total within-cluster:  1021.60895729187 
      SSE Total         :  3039.20264170532 


Another way of investigating how well the clustering was performed, is to tabulate the distribution of the classes across the clusters.

In [11]:
table(MouseDataClean$class, clusters$cluster)

        
          1  2  3  4  5  6  7  8
  c-CS-m  0  0  9  3 12  8  0 13
  c-CS-s  6  0  0  0 26  9  0 34
  c-SC-m  0  0  0 19  0  7 34  0
  c-SC-s 27 27  0  0  0 13  8  0
  t-CS-m 17  0 22  8 12 17  0 14
  t-CS-s  6  0 10  1 12 12  0 34
  t-SC-m  0 16  0 14  0  0 30  0
  t-SC-s  0  5  9 21  0 19  0 18

From the table it is obvious that the distribution is far from perfect, with classes `t-CS-m` and `t-CS-m` being almost uniformly distributed across the clusters. A selection of a different algorithm in the `kmeans()` function, as well as the different parameters (number of clusters, number of random starts) may improve this outcome.

## Hierarchical clustering
In order to perform hierarchical clustering, we need to select the following parameters:
1. The distance metric to be applied to the vectors. Choices are:  
  - "euclidean" *(default)*
  - "maximum"
  - "manhattan"
  - "canberra"
  - "binary"
  - "minkowski"


2. The clustering method for joining similar vectors together. Choice are:  
  - "complete" *(default)*
  - "single"
  - "ward.D" and "ward.D2"
  - "average", which is UPGMA 
  - "mcquitty", which is WPGMA
  - "median", which is WPGMC
  - "centroid", which is UPGMC
  
To check the documentation of the `hclust` and `dist` functions use the command `?hclust` and `?dist` respectively.

In [12]:
# Construct the dissimilarity structure for the dataset
distanceMouseData <- dist(MouseData, method = "euclidean");

# perform the hierarchical clustering with "average" as the method
hc<-hclust(distanceMouseData, method="average");

Finally, as done previously in the K-Means process, print the produced tree in a PNG file. In order to better visualize the distribution of the classes across the clusters, we will use the `` library.

In [13]:
library(dendextend)

png("HierarchicalClusteringMouseProteinExpression.png",     # create PNG for the plot        
    width     = 8000,                                       # set the width of the image in pixels
    height    = 6000,                                       # set the height of the image in pixels
    res       = 300,                                        # set the resolutions to 300 pixels per inch
    pointsize = 5);                                         # set the size of any letters/text

# Convert the hierarchical clustering to a dendrogram (just for visualisation)
dend <- as.dendrogram(hc)

# Create the coding for each class
groupCodes <- c(rep("cCSs", nrow(MouseData_cCSs)), rep("cCSm", nrow(MouseData_cCSm)), rep("cSCs", nrow(MouseData_cSCs)), rep("cSCm", nrow(MouseData_cSCm)),
                rep("tCSs", nrow(MouseData_tCSs)), rep("tCSm", nrow(MouseData_tCSm)), rep("tSCs", nrow(MouseData_tSCs)), rep("tSCm", nrow(MouseData_tSCm)));

# Assign different colors to different classes
colorCodes <- c(cCSs="red", cCSm="orange", cSCs="yellow", cSCm="purple",
                tCSs="blue", tCSm="darkgreen", tSCs="darkgrey", tSCm="green");

# Assigning the labels of dendrogram object with new colors:
labels_colors(dend) <- colorCodes[groupCodes][order.dendrogram(dend)]

# Plotting the new dendrogram
plot(dend)

legend("topright",                                                                    # location of the legend on the plot
       legend = c("cCSs", "cCSm", "cSCs", "cSCm", "tCSs", "tCSm", "tSCs", "tSCm"),    # category labels
       col = colorCodes,                                                              # color codes
       lty= 1,                                                                        # line style
       lwd = 10                                                                       # line width
);

dev.off()

Warning message:
: package 'dendextend' was built under R version 3.2.5
Welcome to dendextend version 1.1.8

Type ?dendextend to access the overall documentation and
browseVignettes(package = 'dendextend') for the package vignette.
You can execute a demo of the package via: demo(dendextend)

More information is available on the dendextend project web-site:
https://github.com/talgalili/dendextend/

Contact: <tal.galili@gmail.com>
Suggestions and bug-reports can be submitted at: https://github.com/talgalili/dendextend/issues

			To suppress this message use:
			suppressPackageStartupMessages(library(dendextend))


Attaching package: 'dendextend'

The following object is masked from 'package:dplyr':

    %>%

The following object is masked from 'package:stats':

    cutree



pdf 
  2

The output of the previous command is the following figure:

![image](HierarchicalClusteringMouseProteinExpression.png)

Again, it is evident that the distribution of classes in the different clusters is not optimal.